In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
from joblib import Parallel, delayed
from tqdm import tqdm
from itertools import product
from itertools import permutations
from itertools import combinations
from pyEDM import *
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import Ridge
from sklearn.ensemble import HistGradientBoostingRegressor
import time
import os
import math
import random
from scipy.stats import ttest_ind

from IPython.display import display, HTML
display(HTML('<style>.container { width:90% !important; }</style>'))

import warnings
warnings.filterwarnings("ignore", 
    message="A worker stopped while some jobs were given to the executor.",
    module="joblib.externals.loky.process_executor")

In [2]:
def get_block(data, num_lags=1, tau=1):
    ''' Get a dataframe with all the possible valid lags of the variables. '''
    
    backward_lags = pd.concat([data[var].shift(lag*tau).rename(f'{var}(t-{lag*tau})') for lag in range(num_lags+1) for var in data.columns], axis=1)
    forward_lags  = pd.concat([data[var].shift(-1*lag*tau).rename(f'{var}(t+{lag*tau})') for lag in range(1,num_lags+1) for var in data.columns], axis=1)
    block = pd.concat([backward_lags, forward_lags], axis=1)

    return block

In [3]:
def get_xmap_results_smap(block, target, embeddings, Tp, theta, lib, pred):
    '''Function to do exhaustive search of embeddings.'''
    
    def compute_rho(block, target, embedding, Tp, theta, lib, pred):
        xmap = SMap(dataFrame=block, target=target, columns=embedding, Tp=Tp, theta=theta, embedded=True, lib=lib, pred=pred, noTime=True)
        rho = xmap['predictions'][['Observations', 'Predictions']].corr().iloc[0,1]
        return embedding, xmap['predictions'], rho

    xmap_results = pd.DataFrame(columns=['embedding', 'rho'])
    xmap_results = Parallel(n_jobs=-1)(delayed(compute_rho)(block, target, embedding, Tp, theta, lib, pred) for embedding in embeddings)
    xmap_results = pd.DataFrame(xmap_results, columns=['embedding', 'result', 'rho'])
    xmap_results = xmap_results.sort_values(by='rho', ascending=False).reset_index(drop=True)
    
    return xmap_results

In [59]:
# Multiview Cross-Mapping Function

def MVCM(block, target, xmap_results, Tp, gap_radius, theta, lib, pred, E, k, self_weight):
    
    # Get lib and pred indices, adjusted to match pyEDM
    lib_start, lib_end = map(int, lib.split())
    pred_start, pred_end = map(int, pred.split())
    lib_start -= 1; lib_end -= 1
    pred_start -= 1; pred_end -= 1
    
    if Tp > 0:
        pred_end += Tp
    elif Tp < 0:
        pred_start -= -1 * Tp
    
    # If k > number of system views, return NaNs as the filtered timeseries
    if k > len(xmap_results):
        filtered_timeseries = pd.DataFrame([np.nan] * len(xmap_results.loc[0,'result']['Predictions']))
        print(f'K={k} is bigger than number of system views')
        return filtered_timeseries
    
    filter_input = pd.DataFrame()
    filter_input = pd.concat([xmap_results.loc[i,'result']['Predictions'] for i in range(0,k)], axis=1)
    filter_input.index = block.loc[pred_start:pred_end,:].index
    
    self = block.loc[pred_start:pred_end,f'{target}(t-0)']
    self.index = range(pred_start,pred_end+1)
    filter_input['self'] = self
    filter_input['vals_to_avg'] = filter_input.apply(lambda row: row.tolist(), axis=1)
    
    # Get weights based on cross-map skill of embeddings
    weights = xmap_results.loc[:k-1,'rho'].tolist()
    weights = [x if x >= 0 else 0 for x in weights]                  # Make negative weights 0
    
    if np.sum(weights) > 0:
        weights = [(1 - self_weight/100)*(weight/np.sum(weights)) for weight in weights]
    else:
        weights = [(1 - self_weight/100)*(1/len(weights)) for weight in weights]
    
    weights = weights + [self_weight/100]
    filter_input['weights'] = [weights] * len(filter_input)
    
    # Get filtered values
    vals_to_avg = np.array(filter_input['vals_to_avg'].tolist())
    weights = np.array(filter_input['weights'].tolist())

    filter_input['filtered_points'] = np.nansum(vals_to_avg * weights, axis=1)
    
    filtered_timeseries = filter_input[['filtered_points']].copy()
    
    # Make sure filtered values are positive
    filtered_timeseries[filtered_timeseries<0] = 0
    
    return filtered_timeseries

In [5]:
def optimize_parameters_MVCM(block, target, all_xmap_results, Tp, gap_radius, lib, pred, E_list, k_list, theta_list):
    
    # Get lib and pred indices, adjusted to match pyEDM
    lib_start, lib_end = map(int, lib.split())
    pred_start, pred_end = map(int, pred.split())
    lib_start -= 1; lib_end -= 1
    pred_start -= 1; pred_end -= 1
    
    # Optimize parameters using a self_weight of 0 until a self_weight is chosen at the end
    self_weight = 0
    
    # Choose the E, k, and theta that give the best multiview cross-map prediction of the observed data with a self_weight of 0

    xmap_results_dict = {}
    
    # Get multiview cross-map predictions for E, k, and theta combinations
    mvcm_results = pd.DataFrame(columns=['E', 'k', 'theta', 'rho', 'xmap_results', 'noisy_and_filtered'])
    
    total_iterations = len(list(product(E_list, theta_list, k_list)))
    #with tqdm(total=total_iterations) as pbar:
    for E, theta in product(E_list, theta_list):

        # Get random embeddings and their cross-map skill
        xmap_results = {k: v for k, v in all_xmap_results.items() if (k.split('_')[0] == target) & 
                                                            (k.split('_')[1] == lib) &
                                                            (k.split('_')[2] == str(E)) &
                                                            (k.split('_')[3] == str(theta))}
        key = list(xmap_results.keys())[0]
        xmap_results = xmap_results[key]
        xmap_results_dict['{0}_{1}'.format(E, theta)] = xmap_results

        # Get multiview cross-map predictions for ks in k_list 
        for k in k_list:

            filtered = MVCM(block, target, xmap_results_dict[f'{str(E)}_{str(theta)}'], Tp, gap_radius, theta, lib, pred, E, k, self_weight)

            # Align indices of noisy target with indices of filtered_timeseries
            noisy_target = block.loc[pred_start:pred_end,f'{target}(t-0)']
            noisy_and_filtered = pd.concat([noisy_target, filtered], axis=1)
            noisy_and_filtered.columns = [f'noisy_{target}', f'filtered_{target}']
            rho = noisy_and_filtered.corr().iloc[0,1]
            mvcm_results.loc[len(mvcm_results)] = [E, k, theta, rho, xmap_results, noisy_and_filtered]
                #pbar.update(1)

    mvcm_results = mvcm_results.sort_values(by='rho', ascending=False).reset_index(drop=True)
    
    E = int(mvcm_results.loc[0,'E'])
    k = int(mvcm_results.loc[0,'k'])
    theta = int(mvcm_results.loc[0,'theta'])
    
    return E, k, theta, mvcm_results

In [6]:
'''
HAB_data = pd.read_csv('Data/data_w_gaps_and_wind.csv', index_col=0)#.iloc[304:612] RANGE w/o missing values
HAB_data.columns = HAB_data.columns.str.replace(' ', '_')

# Put columns in alphabetical order
sorted_columns = sorted(HAB_data.columns)
HAB_data = HAB_data[sorted_columns]

# Make indices integers and save mapping to dates
#date_to_int_map = {i: date for i, date in enumerate(HAB_data.index)}
#HAB_data.index = range(len(HAB_data))
HAB_data.index = HAB_data.index.astype(int)

target = 'Avg_Chloro_(mg/m3)'

#HAB_data = HAB_data.drop(['Nitrite_(uM)','Nitrate_(uM)'],axis=1)

#HAB_data = HAB_data[['Avg_Chloro_(mg/m3)','Nitrate_(uM)','Nitrite_(uM)','Phosphate_(uM)','Silicate_(uM)']]
HAB_data = HAB_data[['Avg_Chloro_(mg/m3)','Phosphate_(uM)','Silicate_(uM)']]
#HAB_data = HAB_data.drop('Nitrite_(uM)',axis=1)

block = get_block(HAB_data, num_lags=50, tau=1)
block
#HAB_data
'''

"\nHAB_data = pd.read_csv('Data/data_w_gaps_and_wind.csv', index_col=0)#.iloc[304:612] RANGE w/o missing values\nHAB_data.columns = HAB_data.columns.str.replace(' ', '_')\n\n# Put columns in alphabetical order\nsorted_columns = sorted(HAB_data.columns)\nHAB_data = HAB_data[sorted_columns]\n\n# Make indices integers and save mapping to dates\n#date_to_int_map = {i: date for i, date in enumerate(HAB_data.index)}\n#HAB_data.index = range(len(HAB_data))\nHAB_data.index = HAB_data.index.astype(int)\n\ntarget = 'Avg_Chloro_(mg/m3)'\n\n#HAB_data = HAB_data.drop(['Nitrite_(uM)','Nitrate_(uM)'],axis=1)\n\n#HAB_data = HAB_data[['Avg_Chloro_(mg/m3)','Nitrate_(uM)','Nitrite_(uM)','Phosphate_(uM)','Silicate_(uM)']]\nHAB_data = HAB_data[['Avg_Chloro_(mg/m3)','Phosphate_(uM)','Silicate_(uM)']]\n#HAB_data = HAB_data.drop('Nitrite_(uM)',axis=1)\n\nblock = get_block(HAB_data, num_lags=50, tau=1)\nblock\n#HAB_data\n"

In [7]:
HAB_data = pd.read_csv('Data/data_w_gaps_and_wind.csv', index_col=0).iloc[:800] 
HAB_data = HAB_data.reset_index()
HAB_data.columns = HAB_data.columns.str.replace(' ', '_')

# Put columns in alphabetical order
sorted_columns = sorted(HAB_data.columns)
HAB_data = HAB_data[sorted_columns]
HAB_data = HAB_data.set_index('time')
target = 'Avg_Chloro'

# Make indices integers and save mapping to dates
#date_to_int_map = {i: date for i, date in enumerate(HAB_data.index)}
#HAB_data.index = range(len(HAB_data))

HAB_data
block = get_block(HAB_data, num_lags=50, tau=1)
block

,AVG_DENS_kgm3(t-0),AVG_SAL_PSU(t-0),AVG_TEMP_C(t-0),Avg_Chloro(t-0),BOT_DENS_kgm3(t-0),BOT_SAL_PSU(t-0),BOT_TEMP_C(t-0),Nitrate(t-0),Nitrite(t-0),Phosphate(t-0),...,BOT_SAL_PSU(t+50),BOT_TEMP_C(t+50),Nitrate(t+50),Nitrite(t+50),Phosphate(t+50),SURF_DENS_kgm3(t+50),SURF_SAL_PSU(t+50),SURF_TEMP_C(t+50),Silicate(t+50),WSPD(t+50)
time,,,,,,,,,,,,,,,,,,,,,
0,1024.129374,33.695,18.10,1.82,1024.620655,33.62,15.9,0.43,0.03,0.24,...,33.66,19.3,0.37,0.0,0.28,1023.818956,33.70,19.4,6.3,4.591667
7,1024.289049,33.640,17.35,2.71,1024.391719,33.62,16.9,0.83,0.04,0.37,...,33.67,20.0,0.34,0.0,0.24,1023.686670,33.73,20.0,5.4,3.308333
13,1023.563298,33.660,20.25,0.99,1024.026040,33.58,18.3,0.45,0.00,0.24,...,33.66,20.6,0.32,0.0,0.28,1023.525345,33.69,20.5,4.2,3.883333
20,1023.740812,33.650,9.90,1.23,1023.780535,33.62,19.4,0.34,0.00,0.17,...,33.52,13.6,0.71,0.0,0.32,1023.998606,33.54,18.2,4.7,2.491667
28,1023.004896,33.710,11.25,2.41,1023.000725,33.69,22.5,0.29,0.00,0.12,...,33.48,17.2,0.17,0.0,0.26,1023.662548,33.56,19.6,0.0,2.316667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5565,1024.078066,33.185,16.75,1.52,1024.389556,33.17,15.4,0.44,0.00,0.26,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5572,1023.529663,33.320,19.45,2.67,1023.676262,33.28,18.8,0.00,0.00,0.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5579,1023.657238,33.255,18.70,1.60,1024.073813,33.20,16.9,0.25,0.00,0.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
print(HAB_data.isna().sum())

AVG_DENS_kgm3     94
AVG_SAL_PSU       91
AVG_TEMP_C        92
Avg_Chloro        10
BOT_DENS_kgm3     91
BOT_SAL_PSU       88
BOT_TEMP_C        91
Nitrate            0
Nitrite            0
Phosphate          0
SURF_DENS_kgm3    52
SURF_SAL_PSU      50
SURF_TEMP_C       50
Silicate           0
WSPD              32
dtype: int64


In [9]:
def get_block(data, num_lags=1, tau=1):
    ''' Get a dataframe with all the possible valid lags of the variables. '''
    
    backward_lags = pd.concat([data[var].shift(lag*tau).rename(f'{var}(t-{lag*tau})') for lag in range(num_lags+1) for var in data.columns], axis=1)
    forward_lags  = pd.concat([data[var].shift(-1*lag*tau).rename(f'{var}(t+{lag*tau})') for lag in range(1,num_lags+1) for var in data.columns], axis=1)
    block = pd.concat([backward_lags, forward_lags], axis=1)
    
    return block

In [10]:
def ccm(interaction, block, E_list, tau_list, theta_list, Tp, sample=50, sig=0.05):
    #solver = HistGradientBoostingRegressor() #TRYING DIFFERNT SOLVER TO ENSURE CONVERGENCE



    print(interaction)
    lib = f'1 {len(block)}'
    
    # Get dataframe with two species of interest
    A = interaction[0]; B = interaction[1]
    df = block[[f'{A}(t-0)', f'{B}(t-0)']]
    
    driver = f'{A}(t-0)'    
    default_output = {
        'target (driver)': A,
        'lib (driven)': B,
        'E': None,
        'tau': None,
        'theta': None,
        'E_tau_theta_results': None,
        'ccm_value': None,
        'convergence_p_value': None,
        'correlation': None
    }
    
    E_tau_theta_results = pd.DataFrame(columns = ['E', 'tau', 'theta', 'rho'])
    for E, tau, theta in list(product(E_list, tau_list, theta_list)):
        driven_embedded = [f'{B}(t{i})' if i < 0 else f'{B}(t-{i})' for i in range(E * tau, 1)]
        driven_embedded = driven_embedded[::tau][:E]
        try:
            c = SMap(dataFrame=block, target=driver, columns=driven_embedded, embedded=True, Tp=Tp, theta=theta, lib=lib, pred=lib, noTime=True)
        except:
            print(f"{interaction} did not converge")
            return default_output
        c = c['predictions'][['Observations', 'Predictions']]
        rho = c.corr().iloc[0,1]
        E_tau_theta_results.loc[len(E_tau_theta_results)] = [E, tau, theta, rho]
    E_tau_theta_results = E_tau_theta_results.apply(pd.to_numeric, errors='coerce')

    # Assign E, tau, and theta to be the optimal E, tau, and theta
    ccm_value = E_tau_theta_results['rho'].max()
    E = int(E_tau_theta_results.loc[np.where(E_tau_theta_results.rho==ccm_value),'E'].item())
    tau = int(E_tau_theta_results.loc[np.where(E_tau_theta_results.rho==ccm_value),'tau'].item())
    theta = int(E_tau_theta_results.loc[np.where(E_tau_theta_results.rho==ccm_value),'theta'].item())
        
    # Get convergence p-value
    try:
        convergence_p_value = get_convergence_p_value(block, sample, A, B, E, Tp, tau, theta)
    except:
        print(f"{interaction} did not converge")
        return default_output

    # Preparing Output
    output = {
        'target (driver)': A,
        'lib (driven)': B,
        'E': E,
        'tau': tau,
        'theta': theta,
        'E_tau_theta_results': E_tau_theta_results,
        'ccm_value': ccm_value,
        'convergence_p_value': convergence_p_value,
        'correlation': df.corr().iloc[0,1]
    }

    return output

def get_convergence_p_value(df, sample, A, B, E, Tp, tau, theta):
    # Get convergence p-value for CCM (one-tailed t-test on cross-map values using 20% and 50% library sizes)
    # H0: μ_20% ≥ μ_50%
    # HA: μ_20% < μ_50%
    # If p < 0.05, the 20% library size trials have a rho that is significantly smaller than the 50% library trials  
    
    libsize1 = int(np.ceil(df.shape[0]/5))   # 20% of the full library size
    libsize2 = int(np.ceil(df.shape[0]/2))   # 50% of the full library size
    
    max_iterations = 10 * sample
    
    # Get list of rhos for libsize1
    rhos1 = []; iteration_count = 0
    while len(rhos1) < sample and iteration_count < max_iterations:
        start = np.random.randint(libsize1, len(df))
        library = [start - libsize1, start]
        data_subset = df.iloc[library[0]:library[1]]
        lib = f'{library[0]+1} {library[1]+1}'
        driver = f'{A}(t-0)'
        driven_embedded = [f'{B}(t{i})' if i < 0 else f'{B}(t-{i})' for i in range(E * tau, 1)]
        driven_embedded = driven_embedded[::tau][:E]
        c = SMap(dataFrame=block, target=driver, columns=driven_embedded, embedded=True, Tp=Tp, theta=theta, lib=lib, pred=lib, noTime=True)
        c = c['predictions'][['Observations', 'Predictions']]
        rho1 = c.corr().iloc[0,1]
        if not np.isnan(rho1):
            rhos1.append(rho1)
        iteration_count += 1
        
    # Get list of rhos for libsize2
    rhos2 = []; iteration_count = 0
    while len(rhos2) < sample and iteration_count < max_iterations:
        start = np.random.randint(libsize2, len(df))
        library = [start - libsize2, start]
        data_subset = df.iloc[library[0]:library[1]]
        lib = f'{library[0]+1} {library[1]+1}'
        driver = f'{A}(t-0)'
        driven_embedded = [f'{B}(t{i})' if i < 0 else f'{B}(t-{i})' for i in range(E * tau, 1)]
        driven_embedded = driven_embedded[::tau][:E]
        c = SMap(dataFrame=block, target=driver, columns=driven_embedded, embedded=True, Tp=Tp, theta=theta, lib=lib, pred=lib, noTime=True)
        c = c['predictions'][['Observations', 'Predictions']]
        rho2 = c.corr().iloc[0,1]
        if not np.isnan(rho2):
            rhos2.append(rho2)
        iteration_count += 1
    
    convergence_t_stat, convergence_p_value = ttest_ind(rhos1, rhos2, alternative='less')
    
    return convergence_p_value


In [11]:
E_list = range(2,13)
tau_list = [-1,-2,-3] #can try more taus
theta_list = [0,0.1,0.5,1,2,3,4,5,6,7,8,9]
Tp = 0
exclusion_radius = 0

all_ccm_results = pd.DataFrame()
interactions = list(permutations(HAB_data.columns.tolist(),2))
target_interactions = [pair for pair in interactions if target in pair]

interaction = target_interactions[0]
print(f'There are {len(target_interactions)} interactions')

results = Parallel(n_jobs=-1)(
    delayed(ccm)(interaction, block, E_list, tau_list, theta_list, Tp) for interaction in target_interactions)
results_df = pd.DataFrame(results)

There are 28 interactions


('AVG_DENS_kgm3', 'Avg_Chloro')
('AVG_TEMP_C', 'Avg_Chloro')
('Avg_Chloro', 'AVG_SAL_PSU')
('AVG_SAL_PSU', 'Avg_Chloro')
('Avg_Chloro', 'BOT_SAL_PSU')
('Avg_Chloro', 'AVG_TEMP_C')
('Avg_Chloro', 'AVG_DENS_kgm3')
('Avg_Chloro', 'BOT_TEMP_C')
('Avg_Chloro', 'BOT_DENS_kgm3')
('Avg_Chloro', 'Nitrate')
('Avg_Chloro', 'Nitrite')
('Avg_Chloro', 'AVG_DENS_kgm3') did not converge
('Avg_Chloro', 'Phosphate')
('Avg_Chloro', 'AVG_TEMP_C') did not converge
('Avg_Chloro', 'SURF_DENS_kgm3')
('Avg_Chloro', 'SURF_SAL_PSU')
('Avg_Chloro', 'SURF_TEMP_C')
('Avg_Chloro', 'Silicate')
('Avg_Chloro', 'WSPD')
('BOT_DENS_kgm3', 'Avg_Chloro')
 ** On entry to DLASCL parameter number  4 had an illegal value
 ** On entry to DLASCL parameter number  4 had an illegal value
('AVG_DENS_kgm3', 'Avg_Chloro') did not converge
('BOT_SAL_PSU', 'Avg_Chloro')


/opt/miniconda3/envs/pyedm_env/lib/python3.9/site-packages/pyEDM/SMap.py:119: RuntimeWarning: divide by zero encountered in divide
  distRowScale = self.theta / distRowMean
/opt/miniconda3/envs/pyedm_env/lib/python3.9/site-packages/pyEDM/SMap.py:120: RuntimeWarning: invalid value encountered in multiply
  W = exp( -distRowScale[:,None] * self.knn_distances )


 ** On entry to DLASCL parameter number  4 had an illegal value
 ** On entry to DLASCL parameter number  4 had an illegal value
('Avg_Chloro', 'Nitrite') did not converge
('BOT_TEMP_C', 'Avg_Chloro')
('Avg_Chloro', 'SURF_TEMP_C') did not converge
('Nitrate', 'Avg_Chloro')
('Nitrite', 'Avg_Chloro')
('Phosphate', 'Avg_Chloro')
('SURF_DENS_kgm3', 'Avg_Chloro')
('SURF_SAL_PSU', 'Avg_Chloro')
('SURF_TEMP_C', 'Avg_Chloro')
('Silicate', 'Avg_Chloro')
('WSPD', 'Avg_Chloro')
 ** On entry to DLASCL parameter number  4 had an illegal value
 ** On entry to DLASCL parameter number  4 had an illegal value
('Nitrite', 'Avg_Chloro') did not converge
 ** On entry to DLASCL parameter number  4 had an illegal value
 ** On entry to DLASCL parameter number  4 had an illegal value
('Phosphate', 'Avg_Chloro') did not converge
 ** On entry to DLASCL parameter number  4 had an illegal value
 ** On entry to DLASCL parameter number  4 had an illegal value
('Nitrate', 'Avg_Chloro') did not converge
 ** On entry t

In [17]:
# Get CCM results that show convergence (convergence p-value < 0.05)

ccm_cutoff = 0.0

significant_results = results_df[results_df.convergence_p_value<0.16]#usually 0.05
significant_results = significant_results.sort_values(by='ccm_value', ascending=False)
significant_results = significant_results[['target (driver)', 'lib (driven)', 'E', 'tau', 'theta', 'ccm_value']].reset_index(drop=True)

display(significant_results[significant_results.ccm_value>ccm_cutoff])

# Choose system variables where the CCM value to or from the target is > ccm_cutoff
system_variables = significant_results[significant_results.ccm_value > ccm_cutoff]
system_variables = system_variables[['target (driver)', 'lib (driven)']].values.flatten().tolist()
system_variables = list(set(system_variables))
print('system variables: ')
display(sorted(system_variables))

,target (driver),lib (driven),E,tau,theta,ccm_value
0,BOT_TEMP_C,Avg_Chloro,12.0,-3.0,7.0,0.357505
1,Avg_Chloro,BOT_TEMP_C,2.0,-2.0,9.0,0.199308
2,AVG_TEMP_C,Avg_Chloro,12.0,-3.0,3.0,0.191026
3,Avg_Chloro,BOT_DENS_kgm3,3.0,-2.0,1.0,0.123969


system variables: 


['AVG_TEMP_C', 'Avg_Chloro', 'BOT_DENS_kgm3', 'BOT_TEMP_C']

In [18]:
results_df

,target (driver),lib (driven),E,tau,theta,E_tau_theta_results,ccm_value,convergence_p_value,correlation
0,AVG_DENS_kgm3,Avg_Chloro,NaN,NaN,NaN,None,NaN,NaN,NaN
1,AVG_SAL_PSU,Avg_Chloro,12.0,-3.0,1.0,E tau theta rho 0 2.0 -1.0...,0.226445,0.999974,0.018325
2,AVG_TEMP_C,Avg_Chloro,12.0,-3.0,3.0,E tau theta rho 0 2.0 -1.0...,0.191026,0.000040,0.022844
3,Avg_Chloro,AVG_DENS_kgm3,NaN,NaN,NaN,None,NaN,NaN,NaN
4,Avg_Chloro,AVG_SAL_PSU,3.0,-3.0,2.0,E tau theta rho 0 2.0 -1.0...,0.429137,0.986081,0.018325
5,Avg_Chloro,AVG_TEMP_C,NaN,NaN,NaN,None,NaN,NaN,NaN
6,Avg_Chloro,BOT_DENS_kgm3,3.0,-2.0,1.0,E tau theta rho 0 2.0 -1.0...,0.123969,0.019258,0.125859
7,Avg_Chloro,BOT_SAL_PSU,3.0,-3.0,2.0,E tau theta rho 0 2.0 -1.0...,0.432986,0.692445,0.035266
8,Avg_Chloro,BOT_TEMP_C,2.0,-2.0,9.0,E tau theta rho 0 2.0 -1.0...,0.199308,0.150631,-0.117058
9,Avg_Chloro,Nitrate,2.0,-3.0,9.0,E tau theta rho 0 2.0 -1.0...,0.212076,0.885963,-0.020844


In [19]:
def get_valid_lags_tau(block, target, Tp, tau, num_lags, exclusion_radius, system_variables):
    
    # Get lags of system variables
    system_variable_lags = []
    for var in system_variables:
        # Get forwards and backwards lag of the system variables
        var_backwards_lags = [f'{var}(t{i})' if i < 0 else f'{var}(t-{i})' for i in range(num_lags * tau, 1)]
        var_backwards_lags = var_backwards_lags[::tau][:num_lags]
        var_forwards_lags  = [f'{var}(t+{i})' for i in range(-(num_lags-1) * tau + 1)]
        var_forwards_lags  = var_forwards_lags[::tau][:num_lags-1]
        var_lags = var_backwards_lags + var_forwards_lags
        system_variable_lags = system_variable_lags + var_lags
    
    # Remove (t-0) lag of target variable from valid_lags
    valid_lags = [x for x in system_variable_lags if x != f'{target}(t-0)']

    # If Tp = 0, remove [-exclusion_radius, exclusion_radius] lags of target variable from valid lags
    if Tp == 0:
        for r in range(-exclusion_radius, exclusion_radius+1):
            if r < 0:
                valid_lags = [x for x in valid_lags if x != f'{target}(t{r})']
            elif r == 0:
                valid_lags = [x for x in valid_lags if x != f'{target}(t-{r})']
            elif r > 0:
                valid_lags = [x for x in valid_lags if x != f'{target}(t+{r})']
                    
    return valid_lags

#target = 'Planktothrix_rubescens'
system_variables = system_variables
Tp = 0
exclusion_radius = 6
num_lags = 2   # Use -3, 0, and +3 lags of each variable
tau = -3

valid_lags = get_valid_lags_tau(block, target, Tp, tau, num_lags, exclusion_radius, system_variables)
valid_lags

['BOT_DENS_kgm3(t-0)',
 'BOT_DENS_kgm3(t-3)',
 'BOT_DENS_kgm3(t+3)',
 'BOT_TEMP_C(t-0)',
 'BOT_TEMP_C(t-3)',
 'BOT_TEMP_C(t+3)',
 'AVG_TEMP_C(t-0)',
 'AVG_TEMP_C(t-3)',
 'AVG_TEMP_C(t+3)']

In [21]:
random_embeddings = {}
for E in range(1,9):
    # Get random embeddings using valid lags
    embeddings = set()
    sample = 100#100000
    max_trials = 100000#10000000
    trials = 0
    while len(embeddings) < sample and trials < max_trials:
        embedding = tuple(random.sample(valid_lags, E))
        sorted_embedding = tuple(sorted(embedding))
        if sorted_embedding not in embeddings:
            embeddings.add(sorted_embedding)
        trials += 1
    embeddings = [list(embedding) for embedding in embeddings]
    random_embeddings['{0}'.format((target, E, Tp, exclusion_radius))] = embeddings
    print(f'E = {E}, # embeddings = {len(embeddings)}')
    
with open('random_embeddings_HAB.pkl', 'wb') as file:
     pickle.dump(random_embeddings, file)

E = 1, # embeddings = 9
E = 2, # embeddings = 36
E = 3, # embeddings = 84
E = 4, # embeddings = 100
E = 5, # embeddings = 100
E = 6, # embeddings = 84
E = 7, # embeddings = 36
E = 8, # embeddings = 9


In [22]:
# Load HAB random embeddings
with open('random_embeddings_HAB.pkl', 'rb') as file:
    HAB_embeddings = pickle.load(file)
HAB_embeddings

{"('Avg_Chloro', 1, 0, 6)": [['AVG_TEMP_C(t+3)'],
  ['BOT_DENS_kgm3(t-0)'],
  ['BOT_TEMP_C(t-0)'],
  ['AVG_TEMP_C(t-0)'],
  ['AVG_TEMP_C(t-3)'],
  ['BOT_TEMP_C(t+3)'],
  ['BOT_TEMP_C(t-3)'],
  ['BOT_DENS_kgm3(t+3)'],
  ['BOT_DENS_kgm3(t-3)']],
 "('Avg_Chloro', 2, 0, 6)": [['BOT_TEMP_C(t-0)', 'BOT_TEMP_C(t-3)'],
  ['AVG_TEMP_C(t-0)', 'BOT_DENS_kgm3(t-3)'],
  ['AVG_TEMP_C(t+3)', 'AVG_TEMP_C(t-0)'],
  ['BOT_DENS_kgm3(t-0)', 'BOT_TEMP_C(t+3)'],
  ['AVG_TEMP_C(t-0)', 'BOT_TEMP_C(t-0)'],
  ['AVG_TEMP_C(t+3)', 'BOT_DENS_kgm3(t+3)'],
  ['AVG_TEMP_C(t-0)', 'BOT_TEMP_C(t-3)'],
  ['AVG_TEMP_C(t+3)', 'BOT_DENS_kgm3(t-3)'],
  ['AVG_TEMP_C(t+3)', 'BOT_TEMP_C(t-0)'],
  ['AVG_TEMP_C(t-0)', 'BOT_DENS_kgm3(t-0)'],
  ['AVG_TEMP_C(t+3)', 'BOT_TEMP_C(t-3)'],
  ['BOT_DENS_kgm3(t-3)', 'BOT_TEMP_C(t-0)'],
  ['AVG_TEMP_C(t-3)', 'BOT_DENS_kgm3(t+3)'],
  ['BOT_DENS_kgm3(t-3)', 'BOT_TEMP_C(t-3)'],
  ['BOT_TEMP_C(t+3)', 'BOT_TEMP_C(t-0)'],
  ['AVG_TEMP_C(t-0)', 'BOT_TEMP_C(t+3)'],
  ['AVG_TEMP_C(t-3)', 'BOT_DENS_k

In [23]:
# Create folder to store xmap results
folder = 'xmap results HAB 100000 random embeddings'
if not os.path.exists(folder):
    os.makedirs(folder)

In [54]:
# Save HAB cross-mapping results
E_list = range(1,9)
theta_list = [1,5,9,15,25,35,45]
Tp = 0
exclusion_radius = 6
self_weight = 0  # self_weight = 0 for gap filling
lib = '1 800' #CHANGE TO LIBRARY SIZE
pred = '1 800'

total_iterations = len(E_list) * len(theta_list)

gapfill_results = {}
parameters = pd.DataFrame(columns=['target', 'noise_level', 'lib', 'pred', 'E', 'theta', 'k', 'rho'])
block = get_block(HAB_data, num_lags=50)

with tqdm(total=total_iterations) as pbar:
    for E, theta in product(E_list, theta_list):

        key = [key for key in HAB_embeddings.keys() if eval(key)[0] == target and eval(key)[1] == E and eval(key)[2] == Tp and eval(key)[3] == exclusion_radius]
        print(key)
        embeddings = HAB_embeddings[key[0]]

        xmap_results = get_xmap_results_smap(block, f'{target}(t-0)', embeddings, Tp, theta, lib, lib)

        file_path = os.path.join(folder, f'xmap_results_{target}_Tp_{Tp}_E_{E}_theta_{theta}.pkl')

        with open(file_path, 'wb') as f:
            pickle.dump(xmap_results, f)

        pbar.update(1)

  0%|          | 0/56 [00:00<?, ?it/s]

["('Avg_Chloro', 1, 0, 6)"]


  2%|▏         | 1/56 [00:02<02:04,  2.27s/it]

["('Avg_Chloro', 1, 0, 6)"]


  4%|▎         | 2/56 [00:02<01:13,  1.36s/it]

...

 96%|█████████▋| 54/56 [01:03<00:00,  2.59it/s]

["('Avg_Chloro', 8, 0, 6)"]


 98%|█████████▊| 55/56 [01:03<00:00,  2.80it/s]

["('Avg_Chloro', 8, 0, 6)"]


100%|██████████| 56/56 [01:03<00:00,  1.14s/it]


In [70]:
# Find optimal parameters

E_list = range(1,9)
theta_list = [1,5,9,15,25,35,45]
k_list = range(1,10)#3,5,10,25,50,75,100,150,200
Tp = 0
exclusion_radius = 6
self_weight = 0  # self_weight = 0 for gap filling
lib = '1 800'
pred = '1 800'

total_iterations = len(E_list) * len(theta_list) * len(k_list)
HAB_data.index = np.arange(0,HAB_data.shape[0])
block = get_block(HAB_data, num_lags=50)

# Get lib and pred indices, adjusted to match pyEDM
lib_start, lib_end = map(int, lib.split())
pred_start, pred_end = map(int, pred.split())
lib_start -= 1; lib_end -= 1
pred_start -= 1; pred_end -= 1

# Get multiview cross-map predictions for E, k, and theta combinations
mvcm_results = pd.DataFrame(columns=['E', 'k', 'theta', 'rho', 'noisy_and_filtered'])
with tqdm(total=total_iterations) as pbar:
    all_xmap_results = {}
    for E, theta in product(E_list, theta_list):
        with open(f'xmap results HAB 100000 random embeddings/xmap_results_{target}_Tp_{Tp}_E_{E}_theta_{theta}.pkl', 'rb') as file:
            xmap_results = pickle.load(file)

        # Get multiview cross-map predictions for ks in k_list 
        for k in k_list:
            #print(block.shape)
            filtered = MVCM(block, target, xmap_results, Tp, exclusion_radius, theta, lib, pred, E, k, self_weight)

            # Align indices of noisy target with indices of filtered_timeseries
            noisy_target = block.loc[pred_start:pred_end,f'{target}(t-0)']
            noisy_and_filtered = pd.concat([noisy_target, filtered], axis=1)
            noisy_and_filtered.columns = [f'noisy_{target}', f'filtered_{target}']
            rho = noisy_and_filtered.corr().iloc[0,1]
            mvcm_results.loc[len(mvcm_results)] = [E, k, theta, rho, noisy_and_filtered]
            pbar.update(1)

100%|██████████| 504/504 [00:01<00:00, 270.32it/s]


In [71]:
# Get xmap results for optimal parameters

E = 8
theta = 45
k = 9

with open(f'xmap results HAB 100000 random embeddings/xmap_results_{target}_Tp_{Tp}_E_{E}_theta_{theta}.pkl', 'rb') as file:
    xmap_results = pickle.load(file)

In [72]:
# Re-rank xmap results for each t after applying exclusion radius to predictions

def re_rank_xmap_results(t, exclusion_radius, xmap_results):
    '''Re-rank xmap results, removing predictions within the exclusion radius for each t.'''
    start_idx = t - exclusion_radius
    end_idx = t + exclusion_radius

    # Re-rank xmap results with exclusion radius removed from predictions
    xmap_results_t = xmap_results.copy(deep=True)
    xmap_results_t['new_rho'] = None

    for idx in xmap_results.index:
        xmap_results_t.at[idx, 'result'] = xmap_results.loc[idx, 'result'].copy(deep=True)
        xmap_results_t.loc[idx,'result'].loc[start_idx:end_idx,'Predictions'] = np.nan
        new_rho = xmap_results_t.loc[idx,'result'][['Observations', 'Predictions']].corr().iloc[0,1]
        xmap_results_t.loc[idx,'new_rho'] = new_rho
    
    return t, xmap_results_t['new_rho']

exclusion_radius = 6

results = Parallel(n_jobs=10)(delayed(re_rank_xmap_results)(t, exclusion_radius, xmap_results) for t in tqdm(HAB_data.index))
results_dict = {t: new_rho for t, new_rho in results}
results_df = pd.DataFrame(results_dict)
xmap_results_for_each_t = pd.concat([xmap_results, results_df.add_prefix('new_rho_')], axis=1)

# with open(f'xmap_results_for_each_t_{target}_Tp_{Tp}_E_{E}_theta_{theta}.pkl', 'wb') as f:
#     pickle.dump(xmap_results_for_each_t, f)

100%|██████████| 800/800 [00:00<00:00, 3585.13it/s]


In [74]:
# Filter points with a six-month exclusion radius on either side

exclusion_radius = 6#6
self_weight = 0
Tp = 0

HAB_gapfill_results = pd.DataFrame(columns=['reconstructed_points'])
with tqdm(total=len(HAB_data)) as pbar:
    for t in HAB_data.index:
        HAB_data_t = HAB_data.copy()
        start_idx = t - exclusion_radius
        end_idx = t + exclusion_radius
        HAB_data_t.loc[start_idx:end_idx,target] = np.nan

        block = get_block(HAB_data_t, num_lags=50)
        
        xmap_results_t = xmap_results_for_each_t.copy(deep=True)
        xmap_results_t = xmap_results_t.sort_values(by=[f'new_rho_{t}'], ascending=False).reset_index(drop=True)
        xmap_results_t = xmap_results_t[['embedding', 'result', f'new_rho_{t}']]
        xmap_results_t.columns = ['embedding', 'result', 'rho']

        filtered = MVCM(block, target, xmap_results_t, Tp, exclusion_radius, theta, lib, pred, E, k, self_weight)
        #print(filtered)
        t_result = filtered.loc[t,f'filtered_points']
        HAB_gapfill_results.loc[t,'reconstructed_points'] = t_result
        pbar.update(1)

100%|██████████| 800/800 [01:02<00:00, 12.90it/s]


In [83]:
sum(HAB_gapfill_results['reconstructed_points']==0)

334